In [2]:
# Imports
import logging
import json
import pathlib
from itertools import chain
import numpy
import os
import pandas

from metrics.enumerators import MethodAveragePrecision
from metrics.pascal_voc_evaluator import (
    get_pascalvoc_metrics,
)

from utils import (
    from_file_to_boundingbox,
    get_filenames_of_path,
)

logger: logging.Logger = logging.getLogger(__name__)

# root directory
FOLDER_PATH = pathlib.Path(os.getcwd()).parent.absolute()
MODULE_PATH = FOLDER_PATH.parent.absolute() / "PyTorch-Object-Detection-Faster-RCNN-Tutorial"
DATA_PATH = MODULE_PATH / "pytorch_faster_rcnn_tutorial" / "data" / "heads"


# input and target files
predictions = get_filenames_of_path(DATA_PATH / "predictions" / "GMT3-488" / "predictions")
predictions.sort()
# test-target files
gt_predictions = get_filenames_of_path(DATA_PATH / "test_targets")
gt_predictions.sort()


# get the gt_boxes from disk  || test-targets
gt_boxes = [
    from_file_to_boundingbox(file_name, groundtruth=True)
    for file_name in gt_predictions
]
# reduce list
gt_boxes = list(chain(*gt_boxes))

pred_boxes = [
    from_file_to_boundingbox(file_name, groundtruth=False)
    for file_name in predictions
]
pred_boxes = list(chain(*pred_boxes))

output = get_pascalvoc_metrics(
    gt_boxes=gt_boxes,
    det_boxes=pred_boxes,
    iou_threshold=0.50,
    method=MethodAveragePrecision.EVERY_POINT_INTERPOLATION,
    generate_table=True,
)

per_class, m_ap = output["per_class"], output["m_ap"]
#print(per_class)



In [3]:
from metrics.bounding_box import BoundingBox
from metrics.enumerators import MethodAveragePrecision

ret = {}
ctr = 0
# Get classes of all bounding boxes separating them by classes
gt_classes_only = []
classes_bbs = {}
for bb in gt_boxes:
    c = bb.get_class_id()
    gt_classes_only.append(c)
    classes_bbs.setdefault(c, {"gt": [], "det": []})
    classes_bbs[c]["gt"].append(bb)
gt_classes_only = list(set(gt_classes_only))
for bb in pred_boxes:
    c = bb.get_class_id()
    confidence_score = bb.get_confidence()
    if c == 'Vehicle' and confidence_score < 0.5:
        continue
    elif c == 'Pedestrian' and confidence_score < 0.2:
        continue
    elif c == 'Bicycle' and confidence_score < 0.2:
        continue
    classes_bbs.setdefault(c, {"gt": [], "det": []})
    classes_bbs[c]["det"].append(bb)
# Precision x Recall is obtained individually by each class
for c, v in classes_bbs.items():
    # Report results only in the classes that are in the GT
    if c not in gt_classes_only:
        continue
    npos = len(v["gt"])
    # sort detections by decreasing confidence
    dects = [
        a
        for a in sorted(v["det"], key=lambda bb: bb.get_confidence(), reverse=True)
    ]
    for n, g in enumerate(gt_boxes):
        for idx_det, det in enumerate(dects):
            iou = BoundingBox.iou(g, det)
            if iou == 0 and g:
                ctr += 1
            else: continue
    print(c,ctr)

NameError: name 'det_boxes' is not defined

In [2]:
labels = ["Pedestrian", "Pedestrian-Violator", "Vehicle", "Vehicle-Violator", "Bicycle", "Bicycle-Violator"]

for label in labels:
    head = per_class[label]
    for key, value in head.items():
        print(key, type(value))
        if isinstance(value, numpy.ndarray):
            head[key] = value.tolist()
        if isinstance(value, MethodAveragePrecision):
            head[key] = str(value)
        if isinstance(value, pandas.core.frame.DataFrame):
            head[key] = value.to_dict()
        if isinstance(value, numpy.int32):
            head[key] = int(value)

    with open(f'my ground truths/{label}.json', 'w') as f:
        json.dump(head, f, indent=4)

precision <class 'numpy.ndarray'>
recall <class 'numpy.ndarray'>
AP <class 'numpy.float64'>
interpolated precision <class 'list'>
interpolated recall <class 'list'>
total positives <class 'int'>
total tp <class 'numpy.float64'>
total fp <class 'numpy.float64'>
total fn <class 'numpy.float64'>
method <enum 'MethodAveragePrecision'>
iou <class 'float'>
table <class 'pandas.core.frame.DataFrame'>
precision <class 'numpy.ndarray'>
recall <class 'numpy.ndarray'>
AP <class 'numpy.float64'>
interpolated precision <class 'list'>
interpolated recall <class 'list'>
total positives <class 'int'>
total tp <class 'numpy.float64'>
total fp <class 'numpy.float64'>
total fn <class 'numpy.float64'>
method <enum 'MethodAveragePrecision'>
iou <class 'float'>
table <class 'pandas.core.frame.DataFrame'>
precision <class 'numpy.ndarray'>
recall <class 'numpy.ndarray'>
AP <class 'numpy.float64'>
interpolated precision <class 'list'>
interpolated recall <class 'list'>
total positives <class 'int'>
total tp <c